# Deliverable 2

- Deliverable 2 will be a NER (Named entity recognition system).


## Overview of the data

url = https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus


Essential info about entities:

```
geo = Geographical Entity
org = Organization
per = Person
gpe = Geopolitical Entity
tim = Time indicator
art = Artifact
eve = Event
nat = Natural Phenomenon
```


In [84]:
import pandas as pd
import numpy as np
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
import skseq
from skseq.sequences import sequence
from skseq.sequences.sequence import Sequence
import csv
import os.path
import scipy
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV
import tqdm
from collections import Counter
import pickle
from sklearn.metrics import confusion_matrix
import skseq.sequences.structured_perceptron as spc

In [85]:
sequence_list_train = pickle.load(open('data/sequence_list_train', 'rb'))
sequence_list_validation = pickle.load(open('data/sequence_list_validation', 'rb'))
sequence_list_test = pickle.load(open('data/sequence_list_test', 'rb'))

# Structured Perceptron- simple FM

## Training Weights of Perceptron

In [86]:
sp = pickle.load(open('data/sp_01', 'rb'))

In [87]:
sp.parameters

array([10. ,  1.5,  1.9, ...,  0. ,  3.6,  1. ])

In [88]:
sp.get_num_states(), sp.get_num_observations()

(17, 25940)

## Predictions

In [89]:
def evaluate_corpus(sequences, sequences_predictions):
    """Evaluate classification accuracy at corpus level, comparing with
    gold standard."""
    total = 0.0
    correct = 0.0
    for i, sequence in enumerate(sequences):
        pred = sequences_predictions[i]
        for j, y_hat in enumerate(pred.y):
            if sequence.y[j] == y_hat:
                correct += 1
            total += 1
    return correct / total

In [90]:
pred_train = pickle.load(open('data/pred_train_1', 'rb'))
pred_dev = pickle.load(open('data/pred_dev_1', 'rb'))
pred_test = pickle.load(open('data/pred_test_1', 'rb'))

In [91]:
# Evaluate and print accuracies
eval_train = evaluate_corpus(sequence_list_train.seq_list, pred_train)
eval_dev = evaluate_corpus(sequence_list_validation.seq_list, pred_dev)
eval_test = evaluate_corpus(sequence_list_test.seq_list, pred_test)
print("SP -  Accuracy Train: %.3f Validation: %.3f Test: %.3f"%(eval_train,eval_dev, eval_test))

SP -  Accuracy Train: 0.959 Validation: 0.696 Test: 0.657


### Confusion matrix

In [92]:
def confusion_matrix_sequence (true, pred):
    true_list=[]
    for i in range(len(true)):
        true_list.extend(true[i].y)
    pred_list=[]
    for i in range(len(pred)):
        pred_list.extend(pred[i].y)

    return confusion_matrix(true_list, pred_list)

In [93]:
# confusion matrix for train dataset
confusion_matrix_sequence(sequence_list_train.seq_list, pred_train)

array([[442183,    269,    347,    614,   1270,    304,    202,     36,
           130,    156,     11,      2,      3,      2,      0,      4,
             0],
       [  1008,  12834,     22,   3445,    307,    454,    120,     66,
           519,      8,     10,      0,      2,      0,      0,      2,
             0],
       [  1719,     53,   8145,     92,     10,      9,      6,      0,
             3,    153,      3,      5,      0,      0,      0,      0,
             0],
       [  1025,    789,      4,   7423,    245,    295,    173,      4,
           200,      6,      7,      8,      6,      0,      0,      0,
             0],
       [   688,     49,      6,    308,   7082,     36,    150,     99,
            27,      0,      4,      3,      1,      0,      0,      1,
             0],
       [   519,    130,      3,    335,    109,   7206,    287,      8,
            18,      2,      3,      1,      2,      0,      0,      0,
             0],
       [   273,      4,      1,   

In [94]:
# confusion matrix for validation dataset
confusion_matrix_sequence(sequence_list_validation.seq_list, pred_dev)

array([[180432,   3980,   3063,   8210,   7801,   8713,   3942,    887,
          3114,    645,     21,     12,     39,     10,      9,     24,
             5],
       [  3176,     52,     38,    173,    359,    238,     64,      4,
            19,      8,      1,      0,      2,      0,      0,      0,
             0],
       [  3043,     95,     41,    175,    524,    154,    180,     12,
            39,     14,      0,      0,      1,      1,      0,      0,
             0],
       [  7390,    631,     74,    368,    171,    531,    153,     11,
            38,     10,      0,      0,      0,      0,      0,      0,
             0],
       [  4427,     86,     23,    146,    141,    116,     51,      4,
            39,     10,      0,      0,      0,      0,      2,      0,
             0],
       [  1507,     47,     13,     48,    133,     36,     39,     18,
            11,      4,      5,      0,      0,      0,      0,      0,
             0],
       [  3556,    143,     75,   

In [95]:
# confusion matrix for test dataset
confusion_matrix_sequence(sequence_list_test.seq_list, pred_test)

array([[170527,   3690,   1507,  10148,   9837,  11164,  11134,    799,
          1901,    623,     27,      3,     54,     17,      7,      6,
             4],
       [  3904,    414,      9,    120,    196,    297,    117,     15,
            12,      5,      0,      0,      0,      0,      0,      0,
             0],
       [  3228,     73,     38,    132,    239,    326,    158,      3,
            22,      5,      0,      0,      1,      1,      7,      0,
             0],
       [  3212,     86,     17,    123,    233,    210,    300,     11,
            52,      9,      0,      0,      4,      3,      0,      0,
             7],
       [  7207,    214,    155,    366,    339,    822,    262,     10,
            66,     28,      0,      0,      1,      0,      0,      0,
             0],
       [  3055,     51,      8,    218,    223,    335,    160,      5,
            15,      4,      1,      0,      0,      0,      0,      0,
             0],
       [  1094,     50,      1,   

### Sentence accuracy 

In [96]:
def sentence_accuracy (true, pred):
    true_list=[]
    for i in range(len(true)):
        true_list.append(true[i].y)
    pred_list=[]
    for i in range(len(pred)):
        pred_list.append(pred[i].y)
    total = 0.0
    correct = 0.0
    for i in zip(true_list, pred_list):

        if list(i[0]) == list(i[1]):
            correct += 1
        total += 1
    return correct/total

In [97]:
#train
sentence_accuracy (sequence_list_train.seq_list, pred_train)

0.5600647329764721

In [98]:
#validation
sentence_accuracy (sequence_list_validation.seq_list, pred_dev)

0.016175231676495365

In [99]:
#test
sentence_accuracy (sequence_list_test.seq_list, pred_test)

0.014096254900325298

# Structured Perceptron- Extended FM

## Building Extended Feature Mapper

In [100]:
from skseq.sequences.id_feature import IDFeatures
from skseq.sequences.id_feature import UnicodeFeatures

# ----------
# Feature Class
# Extracts features from a labeled corpus (only supported features are extracted
# ----------
class ExtendedFeatures(IDFeatures):
    
    def __init__(self, dataset):
        IDFeatures.__init__(self, dataset) #Get all the previous init of the inherited class
        self.additional_features = [] #If empty, use all, if 0, use none
    
    def add_emission_features(self, sequence, pos, y, features):
        x = sequence.x[pos]
        # Get tag name from ID.
        y_name = self.dataset.y_dict.get_label_name(y)

        # Get word name from ID.
        if isinstance(x, str):
            x_name = x
        else:
            x_name = self.dataset.x_dict.get_label_name(x)

        word = str(x_name)
        # Generate feature name.
        feat_name = "id:%s::%s" % (word, y_name)
        # Get feature ID from name.
        feat_id = self.add_feature(feat_name)
        # Append feature.
        if feat_id != -1:
            features.append(feat_id)
        
        # Suffixes
        max_suffix = 3
        for i in range(max_suffix):
            if len(word) > i+1:
                suffix = word[-(i+1):]
                # Generate feature name.
                feat_name = "suffix:%s::%s" % (suffix, y_name)
                # Get feature ID from name.
                feat_id = self.add_feature(feat_name)
                # Append feature.
                if feat_id != -1:
                    features.append(feat_id)  
                    
                    
        if len(self.additional_features) == 0 or 2 in self.additional_features:
        #If there is a capital letter at the beggining
            if word[0].isupper():
                # Generate feature name.
                feat_name = "upper::%s" % y_name
                # Get feature ID from name.
                feat_id = self.add_feature(feat_name)
                # Append feature.
                if feat_id != -1:
                    features.append(feat_id)

        if len(self.additional_features) == 0 or 3 in self.additional_features:
            #Hyphen
            if str.find(word, "-") != -1:
                # Generate feature name.
                feat_name = "hyphen::%s" % y_name
                # Get feature ID from name.
                feat_id = self.add_feature(feat_name)
                # Append feature.
                if feat_id != -1:
                    features.append(feat_id)

        if len(self.additional_features) == 0 or 4 in self.additional_features:
            #If there the word ends with -ed   
            if str.endswith(word,'ed'):
                # Generate feature name.
                feat_name = "verb_ed::%s" % y_name
                # Get feature ID from name.
                feat_id = self.add_feature(feat_name)
                # Append feature.
                if feat_id != -1:
                    features.append(feat_id)

        if len(self.additional_features) == 0 or 5 in self.additional_features:
            #If there the word ends with -ly      
            if str.endswith(word,'ly'):
                # Generate feature name.
                feat_name = "adverb_ly::%s" % y_name
                # Get feature ID from name.
                feat_id = self.add_feature(feat_name)
                # Append feature.
                if feat_id != -1:
                    features.append(feat_id)


        if len(self.additional_features) == 0 or 6 in self.additional_features:
            #If the word ends with 'ing'
            if str.endswith(word,'ing'):
                # Generate feature name.
                feat_name = "end_ing::%s" % y_name
                # Get feature ID from name.
                feat_id = self.add_feature(feat_name)
                # Append feature.
                if feat_id != -1:
                    features.append(feat_id)
                    
        if len(self.additional_features) == 0 or 7 in self.additional_features:
            #If the word ends with 'or'
            if str.endswith(word,'or'):
                # Generate feature name.
                feat_name = "or_suf::%s" % y_name
                # Get feature ID from name.
                feat_id = self.add_feature(feat_name)
                # Append feature.
                if feat_id != -1:
                    features.append(feat_id)        
        
        if len(self.additional_features) == 0 or 8 in self.additional_features:
            #If the word starts with 'up'
            if str.startswith(word,'up'):
                # Generate feature name.
                feat_name = "up_pref::%s" % y_name
                # Get feature ID from name.
                feat_id = self.add_feature(feat_name)
                # Append feature.
                if feat_id != -1:
                    features.append(feat_id)


        if len(self.additional_features) == 0 or 9 in self.additional_features: 
            #If the word contains a full stop 
            if str.find(word, ".") != -1:
                # Generate feature name.
                feat_name = "points::%s" % y_name
                # Get feature ID from name.
                feat_id = self.add_feature(feat_name)
                # Append feature.
                if feat_id != -1:
                    features.append(feat_id)

        if len(self.additional_features) == 0 or 10 in self.additional_features:      
            #If the word is 'to'
            if word == 'to':
                # Generate feature name.
                feat_name = "to_prep::%s" % y_name
                # Get feature ID from name.
                feat_id = self.add_feature(feat_name)
                # Append feature.
                if feat_id != -1:
                    features.append(feat_id)


        if len(self.additional_features) == 0 or 11 in self.additional_features:
            #If the word is 'of'
            if word == 'of':
                # Generate feature name.
                feat_name = "of_prep::%s" % y_name
                # Get feature ID from name.
                feat_id = self.add_feature(feat_name)
                # Append feature.
                if feat_id != -1:
                    features.append(feat_id)



        if len(self.additional_features) == 0 or 12 in self.additional_features:
            #If the word is 'from'
            if word == 'from':
                # Generate feature name.
                feat_name = "from_prep::%s" % y_name
                # Get feature ID from name.
                feat_id = self.add_feature(feat_name)
                # Append feature.
                if feat_id != -1:
                    features.append(feat_id)
                    

        if len(self.additional_features) == 0 or 13 in self.additional_features:
            #If the word is 'the'
            if word == 'the':
                # Generate feature name.
                feat_name = "the_art::%s" % y_name
                # Get feature ID from name.
                feat_id = self.add_feature(feat_name)
                # Append feature.
                if feat_id != -1:
                    features.append(feat_id)



        if len(self.additional_features) == 0 or 14 in self.additional_features:
            #If the word is 'in'
            if word == 'in':
                # Generate feature name.
                feat_name = "in_prep::%s" % y_name
                # Get feature ID from name.
                feat_id = self.add_feature(feat_name)
                # Append feature.
                if feat_id != -1:
                    features.append(feat_id)


                    
        return features

In [101]:
feature_mapper_2 = ExtendedFeatures(sequence_list_train)
feature_mapper_2.build_features()

## Training Weights of Perceptron

In [102]:
sp = pickle.load(open('data/sp_02', 'rb'))

## Predictions

In [103]:
# Make predictions for the various sequences using the trained model.
pred_train = pickle.load(open('data/pred_train_2', 'rb'))
pred_dev = pickle.load(open('data/pred_dev_2', 'rb'))
pred_test = pickle.load(open('data/pred_test_2', 'rb'))

In [104]:
# Evaluate and print accuracies
eval_train = evaluate_corpus(sequence_list_train.seq_list, pred_train)
eval_dev = evaluate_corpus(sequence_list_validation.seq_list, pred_dev)
eval_test = evaluate_corpus(sequence_list_test.seq_list, pred_test)
print("SP -  Accuracy Train: %.3f Validation: %.3f Test: %.3f"%(eval_train,eval_dev, eval_test))

SP -  Accuracy Train: 0.961 Validation: 0.696 Test: 0.622


### Confusion matrix

In [105]:
# confusion matrix for train dataset
confusion_matrix_sequence(sequence_list_train.seq_list, pred_train)

array([[441987,    303,    298,    752,   1503,    179,    193,     22,
           158,    119,      7,      6,      2,      1,      0,      3,
             0],
       [   634,  14805,     20,   2099,    384,    179,    106,     46,
           514,      5,      4,      0,      1,      0,      0,      0,
             0],
       [  2002,     61,   7865,    101,     13,      3,      6,      0,
             3,    139,      2,      3,      0,      0,      0,      0,
             0],
       [   660,    768,      2,   7898,    352,    121,    152,      2,
           211,      4,      3,      7,      5,      0,      0,      0,
             0],
       [   331,     49,      5,    185,   7708,     10,     59,     75,
            26,      0,      3,      3,      0,      0,      0,      0,
             0],
       [   390,    213,      2,    900,    357,   6403,    293,      7,
            51,      1,      5,      1,      0,      0,      0,      0,
             0],
       [   133,     12,      1,   

In [106]:
# confusion matrix for validation dataset
confusion_matrix_sequence(sequence_list_validation.seq_list, pred_dev)

array([[180508,   6477,   3000,   9809,   9089,   5117,   2211,    686,
          3340,    597,      7,      1,     33,      7,      5,     19,
             1],
       [  3181,     85,     22,    256,    405,    123,     27,      3,
            26,      5,      0,      0,      1,      0,      0,      0,
             0],
       [  3093,    151,     34,    208,    569,     88,     62,      9,
            53,     10,      0,      0,      1,      0,      0,      1,
             0],
       [  7332,    718,     56,    469,    174,    447,    127,      3,
            39,     12,      0,      0,      0,      0,      0,      0,
             0],
       [  4442,    104,     46,    194,    137,     31,     20,      2,
            47,     19,      2,      0,      0,      0,      0,      0,
             1],
       [  1462,     63,     12,     66,    166,     36,     19,      7,
            20,      3,      3,      0,      0,      0,      0,      4,
             0],
       [  3607,    213,     56,   

In [107]:
# confusion matrix for test dataset
confusion_matrix_sequence(sequence_list_test.seq_list, pred_test)

array([[161106,   7085,   1378,  14635,  14313,  13236,   6329,    542,
          2068,    617,     17,      6,     75,     28,      5,      5,
             3],
       [  3716,    394,      9,    207,    392,    257,     70,     11,
            26,      7,      0,      0,      0,      0,      0,      0,
             0],
       [  3501,     98,     25,    158,    221,    139,     52,      1,
            34,      3,      0,      0,      0,      0,      0,      1,
             0],
       [  3239,    176,     17,    213,    282,    178,     88,      7,
            52,      8,      0,      0,      4,      2,      1,      0,
             0],
       [  7222,    276,    114,    475,    579,    570,    137,      6,
            83,      7,      0,      0,      1,      0,      0,      0,
             0],
       [  3097,     97,     11,    285,    312,    194,     55,      5,
            18,      0,      0,      0,      0,      1,      0,      0,
             0],
       [  1122,     51,      0,   

### Sentence accuracy

In [108]:
#train
sentence_accuracy (sequence_list_train.seq_list, pred_train)

0.6023901406697373

In [109]:
#validation
sentence_accuracy (sequence_list_validation.seq_list, pred_dev)

0.016933445661331085

In [110]:
#test
sentence_accuracy (sequence_list_test.seq_list, pred_test)

0.0001668195846192343

# Perceptron evaluation using the testing sentences

In [111]:
p = "The programmers from Barcelona might write a sentence without a spell checker. The programmers from Barchelona cannot write a sentence without a spell checker. Jack London went to Parris. Jack London went to Paris. We never though Microsoft would become such a big company. We never though Microsof would become such a big company. The president of U.S.A though they could win the war The president of the United States of America though they could win the war The king of Saudi Arabia wanted total control. Robin does not want to go to Saudi Arabia. Apple is a great company. I really love apples and oranges."

new_seq = skseq.sequences.sequence.Sequence(x=p.split(), y=[int(0) for w in p.split()])
new_seq

The/0 programmers/0 from/0 Barcelona/0 might/0 write/0 a/0 sentence/0 without/0 a/0 spell/0 checker./0 The/0 programmers/0 from/0 Barchelona/0 cannot/0 write/0 a/0 sentence/0 without/0 a/0 spell/0 checker./0 Jack/0 London/0 went/0 to/0 Parris./0 Jack/0 London/0 went/0 to/0 Paris./0 We/0 never/0 though/0 Microsoft/0 would/0 become/0 such/0 a/0 big/0 company./0 We/0 never/0 though/0 Microsof/0 would/0 become/0 such/0 a/0 big/0 company./0 The/0 president/0 of/0 U.S.A/0 though/0 they/0 could/0 win/0 the/0 war/0 The/0 president/0 of/0 the/0 United/0 States/0 of/0 America/0 though/0 they/0 could/0 win/0 the/0 war/0 The/0 king/0 of/0 Saudi/0 Arabia/0 wanted/0 total/0 control./0 Robin/0 does/0 not/0 want/0 to/0 go/0 to/0 Saudi/0 Arabia./0 Apple/0 is/0 a/0 great/0 company./0 I/0 really/0 love/0 apples/0 and/0 oranges./0 

In [112]:
feature_mapper_2.get_sequence_features(new_seq)

([[0]],
 [[8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8],
  [8]],
 [[105]],
 [[106, 2, 23, 3],
  [36208, 62, 135, 136],
  [867, 270, 868, 869, 870],
  [36092, 310, 311, 36093, 3],
  [2029, 10, 2030, 2031],
  [21757, 2, 445, 2625],
  [94],
  [9623, 2, 140, 504],
  [4258, 10, 453, 557],
  [94],
  [5, 6, 772],
  [6935, 103]

In [113]:
sp.viterbi_decode(new_seq)

(The/0 programmers/0 from/0 Barcelona/1 might/0 write/0 a/0 sentence/0 without/0 a/0 spell/5 checker./6 The/0 programmers/0 from/0 Barchelona/3 cannot/4 write/0 a/0 sentence/0 without/0 a/0 spell/5 checker./6 Jack/0 London/1 went/0 to/0 Parris./5 Jack/6 London/6 went/0 to/0 Paris./5 We/6 never/0 though/0 Microsoft/3 would/0 become/0 such/0 a/0 big/0 company./5 We/6 never/0 though/0 Microsof/0 would/0 become/0 such/0 a/0 big/5 company./6 The/0 president/0 of/0 U.S.A/3 though/0 they/0 could/0 win/0 the/0 war/0 The/0 president/0 of/0 the/0 United/1 States/7 of/7 America/7 though/0 they/0 could/0 win/0 the/0 war/0 The/0 king/0 of/0 Saudi/1 Arabia/7 wanted/0 total/0 control./5 Robin/6 does/0 not/0 want/0 to/0 go/0 to/0 Saudi/3 Arabia./4 Apple/4 is/0 a/0 great/0 company./5 I/6 really/0 love/0 apples/0 and/0 oranges./5 ,
 3143.200000000003)

In [114]:
sp.viterbi_decode(new_seq)[0].to_words(sequence_list_train,
                                       only_tag_translation=True)

'The/O programmers/O from/O Barcelona/B-geo might/O write/O a/O sentence/O without/O a/O spell/B-per checker./I-per The/O programmers/O from/O Barchelona/B-org cannot/I-org write/O a/O sentence/O without/O a/O spell/B-per checker./I-per Jack/O London/B-geo went/O to/O Parris./B-per Jack/I-per London/I-per went/O to/O Paris./B-per We/I-per never/O though/O Microsoft/B-org would/O become/O such/O a/O big/O company./B-per We/I-per never/O though/O Microsof/O would/O become/O such/O a/O big/B-per company./I-per The/O president/O of/O U.S.A/B-org though/O they/O could/O win/O the/O war/O The/O president/O of/O the/O United/B-geo States/I-geo of/I-geo America/I-geo though/O they/O could/O win/O the/O war/O The/O king/O of/O Saudi/B-geo Arabia/I-geo wanted/O total/O control./B-per Robin/I-per does/O not/O want/O to/O go/O to/O Saudi/B-org Arabia./I-org Apple/I-org is/O a/O great/O company./B-per I/I-per really/O love/O apples/O and/O oranges./B-per '

# HMM Model

## prediction

In [115]:
Y_hat = pickle.load(open('data/Y_hat', 'rb'))
Y = pickle.load(open('data/Y', 'rb'))
Y_hat_2 = pickle.load(open('data/Y_hat_2', 'rb'))
Y_2 = pickle.load(open('data/Y_2', 'rb'))
Y_hat_3 = pickle.load(open('data/Y_hat_3', 'rb'))
Y_3 = pickle.load(open('data/Y_3', 'rb'))

In [116]:

def hmm_prediction (Y, Y_hat):
    correct = 0
    total   = 0
    for y,y_hat in zip(Y,Y_hat):
        for y_hat_k, y_k in zip(y,y_hat):
            total +=1
            if y_hat_k == y_k:
                correct +=1
    correct, total
    print("Accuracy posterior decode data", correct/total)

In [117]:
# train
hmm_prediction(Y, Y_hat)

Accuracy posterior decode data 0.7115939936471267


In [118]:
# validation
hmm_prediction(Y_2, Y_hat_2)

Accuracy posterior decode data 0.9754079495500472


In [119]:
# test
hmm_prediction(Y_3, Y_hat_3)

Accuracy posterior decode data 0.9766103547232104
